In [1]:
import tensorflow as tf

Tensorflow, by default, runs code as chunks. So when you do the following, you don't get the answer you'd expect - it gives you a tensor object that hasn't actually been calculated yet.

In [5]:
x1 = tf.constant([1,2,3,4])
x2 = tf.constant([5,6,7,8])
result = tf.multiply(x1, x2)
print(result)

Tensor("Mul_1:0", shape=(4,), dtype=int32)


Note that a constant is an assigned value that does not change. Placeholders are unassigned values that are initialized by the session when you run it. Variables are values that can change.

An interactive session in Tensorflow works more like Python.

In [6]:
x1 = tf.constant([1,2,3,4])
x2 = tf.constant([5,6,7,8])
result = tf.multiply(x1, x2)

sess = tf.Session()
print(sess.run(result))
sess.close()

[ 5 12 21 32]


Or alternatively:

In [7]:
with tf.Session() as sess:
    output = sess.run(result)
    print(output)

[ 5 12 21 32]


note that config=tf.ConfigProto(kwargs) can be used to configure the session

Now we write a function that loads in the data. This reads each file in the trainingand test sets.

In [16]:
import os
import skimage.io

In [17]:
def load_data(data_directory):
    directories = [d for d in os.listdir(data_directory) 
                   if os.path.isdir(os.path.join(data_directory, d))]
    labels = []
    images = []
    for d in directories:
        label_directory = os.path.join(data_directory, d)
        file_names = [os.path.join(label_directory, f) 
                      for f in os.listdir(label_directory) 
                      if f.endswith(".ppm")]
        for f in file_names:
            images.append(skimage.io.imread(f))
            labels.append(int(d))
    return images, labels

In [ ]:
ROOT_PATH = "/your/root/path"
train_data_directory = os.path.join(ROOT_PATH, "Training")
test_data_directory = os.path.join(ROOT_PATH, "Testing")

images, labels = load_data(train_data_directory)

In [18]:
images, labels = load_data("Training")

In [20]:
images

array([[[ 47,  52,  59],
        [ 35,  64,  72],
        [ 53, 104, 112],
        ...,
        [ 75,  82,  60],
        [ 72,  77,  55],
        [ 68,  71,  50]],

       [[ 46,  51,  57],
        [ 38,  66,  74],
        [ 58, 109, 115],
        ...,
        [ 74,  84,  62],
        [ 76,  84,  62],
        [ 76,  83,  61]],

       [[ 44,  50,  56],
        [ 44,  71,  79],
        [ 59, 109, 115],
        ...,
        [ 71,  83,  61],
        [ 74,  84,  63],
        [ 75,  84,  63]],

       ...,

       [[120, 141, 139],
        [119, 144, 138],
        [114, 141, 131],
        ...,
        [ 33,  30,  27],
        [ 32,  30,  28],
        [ 30,  30,  28]],

       [[133, 151, 145],
        [124, 146, 138],
        [111, 137, 127],
        ...,
        [ 31,  30,  25],
        [ 32,  32,  27],
        [ 33,  33,  30]],

       [[139, 158, 147],
        [124, 146, 137],
        [107, 132, 123],
        ...,
        [ 31,  30,  23],
        [ 33,  33,  27],
        [ 35,  36,  31]]

In [22]:
labels[0]

31